In [39]:
###############################
## This script links the flood sum map that was generated in GEE to the corresponding flood frequenices 
## for the Prek Area 
## exüprted from this script: https://code.earthengine.google.com/f69af6afbec2c0a5d43a173a965c67be

In [40]:
# import necessary packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime
import os
import matplotlib.pyplot as plt
from io import StringIO
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import math
from matplotlib.dates import date2num
from pandas import DataFrame

In [41]:
# water level at Koh Kehl 

wl_long = pd.read_csv('wl_KK.csv')
wl_long['date'] = pd.to_datetime(wl_long['date'])
wl_long['date'] = wl_long['date'].dt.strftime('%Y-%m-%d')
wl_long.columns = ['huh', 'date', 'wl_long']
wl_long = wl_long.drop(columns=['huh'])
wl_long = wl_long.set_index('date')
wl_long['wl_long'] = wl_long['wl_long']-1 # <------ -1 correct for elevation 

# for data availability comparison:
wl_long.index = pd.to_datetime(wl_long.index)
wl_long1 = wl_long.resample('d').max()

wl_KK_daily = wl_long1  
wl_KK_daily.columns = ['wl_KK']
wl_original = wl_KK_daily

#wl_KK_daily = wl_KK_daily.rolling(center=False, window=3, min_periods=1).mean().round(2)

#wl_KK_daily.to_csv('wl_KK_smoothed.csv')

wl_KK_daily.head()

,wl_KK
date,
1990-09-05,7.16
1990-09-06,7.20
1990-09-07,7.22
1990-09-08,7.22
1990-09-09,7.21


In [11]:
#### SIMPLE: calculate fixed shifts! 
wl_KK_daily['date'] = wl_KK_daily.index
wl_KK_daily['Sentinel_date_PA'] = wl_KK_daily['date'].shift(-12)
wl_KK_daily['Sentinel_date_2C'] = wl_KK_daily['date'].shift(-25)


In [12]:
wl_KK_daily.head(15)

,wl_KK,date,Sentinel_date_PA,Sentinel_date_2C
date,,,,
1990-09-05,7.16,1990-09-05,1990-09-17,1990-09-30
1990-09-06,7.20,1990-09-06,1990-09-18,1990-10-01
1990-09-07,7.22,1990-09-07,1990-09-19,1990-10-02
1990-09-08,7.22,1990-09-08,1990-09-20,1990-10-03
1990-09-09,7.21,1990-09-09,1990-09-21,1990-10-04
1990-09-10,7.20,1990-09-10,1990-09-22,1990-10-05
1990-09-11,7.18,1990-09-11,1990-09-23,1990-10-06
1990-09-12,7.18,1990-09-12,1990-09-24,1990-10-07
1990-09-13,7.16,1990-09-13,1990-09-25,1990-10-08


In [13]:
wl=wl_KK_daily

d = pd.DataFrame()

wl['year'] = wl['date'].dt.strftime('%Y')


years = ["1990", "1991", "1992", "1993", "1994", "1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]



for y in years: 
    wl_year = wl[(wl['year']==y)]
    max_level = wl_year['wl_KK'].max()
    max_date = wl_year.loc[(wl_year.wl_KK==max_level).idxmax()]['date']
    max_date_PA = wl_year.loc[(wl_year.wl_KK==max_level).idxmax()]['Sentinel_date_PA']
    max_date_2C = wl_year.loc[(wl_year.wl_KK==max_level).idxmax()]['Sentinel_date_2C']
    min_level = wl_year['wl_KK'].min()
    min_date = wl_year.loc[(wl_year.wl_KK==min_level).idxmax()]['date']
    min_date_PA = wl_year.loc[(wl_year.wl_KK==min_level).idxmax()]['Sentinel_date_PA']
    min_date_2C = wl_year.loc[(wl_year.wl_KK==min_level).idxmax()]['Sentinel_date_2C']
    
    temp1 = pd.DataFrame(
            {
                #'year': [y],
                'min_date_KK': [min_date],
                'min_date_PA': [min_date_PA],
                'min_date_2C': [min_date_2C],
                #'min_wl': [min_level],
                'max_date_KK': [max_date],
                'max_date_PA': [max_date_PA],
                'max_date_2C': [max_date_2C],
                #'max_wl': [max_level]
            }
        )
    d = pd.concat([d, temp1])
    
print(d)

  min_date_KK min_date_PA min_date_2C max_date_KK max_date_PA max_date_2C
0  1990-12-31  1991-01-12  1991-01-25  1990-10-23  1990-11-04  1990-11-17
0  1991-04-12  1991-04-24  1991-05-07  1991-09-12  1991-09-24  1991-10-07
0  1992-04-13  1992-04-25  1992-05-08  1992-08-26  1992-09-07  1992-09-20
0  1993-05-03  1993-05-15  1993-05-28  1993-09-19  1993-10-01  1993-10-14
0  1994-05-06  1994-05-18  1994-05-31  1994-09-25  1994-10-07  1994-10-20
0  1995-05-10  1995-05-22  1995-06-04  1995-09-20  1995-10-02  1995-10-15
0  1996-05-03  1996-05-15  1996-05-28  1996-10-02  1996-10-14  1996-10-27
0  1997-05-16  1997-05-28  1997-06-10  1997-08-09  1997-08-21  1997-09-03
0  1998-05-05  1998-05-17  1998-05-30  1998-09-25  1998-10-07  1998-10-20
0  1999-03-12  1999-03-24  1999-04-06  1999-10-05  1999-10-17  1999-10-30
0  2000-04-14  2000-04-26  2000-05-09  2000-09-21  2000-10-03  2000-10-16
0  2001-05-03  2001-05-15  2001-05-28  2001-09-02  2001-09-14  2001-09-27
0  2002-04-23  2002-05-05  2002-05-18 

In [14]:
##### import list of dates of IMAGES manually generated 

dates_simple = pd.read_csv('IE_PA_Simple.csv')
dates_simple['Date'] = pd.to_datetime(dates_simple['Date'])
dates_simple['Date'] = dates_simple['Date'].dt.strftime('%Y-%m-%d')
dates_simple['date'] = dates_simple['Date']

dates_simple = dates_simple.set_index('date')

df = dates_simple.drop(columns='Date')


df.tail(10)



""
date
2016-09-15
2019-09-25
2018-09-25
2017-09-25
2016-09-25
2016-09-29
2017-08-14
2019-09-24
2011-09-17


In [15]:
##### match them to the water levels at KK at the correct date

wl_KK_PA = wl_KK_daily

wl_KK_PA = wl_KK_PA.set_index('Sentinel_date_PA') # set the PA date to index

wl_KK_PA = wl_KK_PA.sort_values('wl_KK') # sort by water level (ascending)

wl_KK_PA = wl_KK_PA[~wl_KK_PA.index.duplicated(keep='first')] # get rid of duplicates

flood_link = df.merge(wl_KK_PA, how='inner', left_index=True, right_index=True)

flood_link = flood_link.drop(columns=['Sentinel_date_2C'])

flood_link = flood_link.sort_values('wl_KK',ascending=False)

#flood_link['image_date'] = flood_link.index

flood_link.index = np.arange(1, len(flood_link) + 1)

flood_link['count_reached'] = flood_link.index

flood_link_original = flood_link

flood_link.head()

,wl_KK,date,year,count_reached
1,6.82,2018-09-03,2018,1
2,6.70,2018-09-13,2018,2
3,6.68,2019-09-12,2019,3
4,6.68,2019-09-13,2019,4
5,6.66,2018-08-19,2018,5


In [16]:
wl_original.head()

,wl_KK,date,Sentinel_date_PA,Sentinel_date_2C,year
date,,,,,
1990-09-05,7.16,1990-09-05,1990-09-17,1990-09-30,1990
1990-09-06,7.20,1990-09-06,1990-09-18,1990-10-01,1990
1990-09-07,7.22,1990-09-07,1990-09-19,1990-10-02,1990
1990-09-08,7.22,1990-09-08,1990-09-20,1990-10-03,1990
1990-09-09,7.21,1990-09-09,1990-09-21,1990-10-04,1990


In [17]:
wl_original['date'] = pd.to_datetime(wl_original['date'])
wl_original['monthday'] = wl_original['date'].dt.strftime('%m-%d')

In [18]:
#### for each value in colum: count how often it has been reached by a certain date in the years since 1990 overall 

### get list of years in that file! 

#years = wl_original.year.unique()

### define function that counts how many times in this time range, 
### a certain value was exceeded by the test date! 

def counting_times_reached(wl):
        list1 = []
        for year in years: 
            year = str(year)
            wl_year = wl_original[(wl_original['year']==year)] # filter for year 
            wl_decide = wl_year[(wl_year['monthday']<=testdate)] # filter before date
            #wl_min = wl_decide1['wl_long'].min()
            #date_min = wl_decide1[(wl_decide1.wl_long==wl_min).idxmax()]['date']  
            #wl_decide = wl_decide1[(wl_decide1['date'] > date_min)] # after dry season min
            check = (wl_decide['wl_KK'] >= wl).any()#.astype(int) # check if present
            list1.append(check)
        total_sum = float(sum(list1))/float(len(years))
        return total_sum
            
            

In [35]:
wl_narrow = wl_original[(wl_original['year']>'1990')]
wl_90s = wl_narrow[(wl_narrow['year']<='2000')]
wl_narrow2 = wl_narrow[(wl_narrow)['year']>'2000']
wl_00s = wl_narrow2[(wl_narrow2['year']<='2010')]
wl_narrow3 = wl_narrow[(wl_narrow)['year']>'2010']
wl_10s = wl_narrow3[(wl_narrow3['year']<='2020')]


wl_pre08 = wl_original[(wl_original['year'] <= '2008')]
wl_post08 = wl_original[(wl_original['year'] > '2008')]

years1 = wl_90s.year.unique()
years2 = wl_00s.year.unique()
years3 = wl_10s.year.unique()

print(years1)
print(years2)
print(years3)


['1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999' '2000']
['2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009' '2010']
['2011' '2012' '2013' '2014' '2015' '2016' '2017' '2018' '2019' '2020']


In [36]:
wl_original.head()

,wl_KK,date,Sentinel_date_PA,Sentinel_date_2C,year,monthday
date,,,,,,
1990-09-05,7.16,1990-09-05,1990-09-17,1990-09-30,1990,09-05
1990-09-06,7.20,1990-09-06,1990-09-18,1990-10-01,1990,09-06
1990-09-07,7.22,1990-09-07,1990-09-19,1990-10-02,1990,09-07
1990-09-08,7.22,1990-09-08,1990-09-20,1990-10-03,1990,09-08
1990-09-09,7.21,1990-09-09,1990-09-21,1990-10-04,1990,09-09


In [37]:
wl_00s.head()

,wl_KK,date,Sentinel_date_PA,Sentinel_date_2C,year,monthday
date,,,,,,
2001-01-01,3.55,2001-01-01,2001-01-13,2001-01-26,2001,01-01
2001-01-02,3.49,2001-01-02,2001-01-14,2001-01-27,2001,01-02
2001-01-03,3.46,2001-01-03,2001-01-15,2001-01-28,2001,01-03
2001-01-04,3.40,2001-01-04,2001-01-16,2001-01-29,2001,01-04
2001-01-05,3.34,2001-01-05,2001-01-17,2001-01-30,2001,01-05


In [38]:
#### implement the function for the OVERALL time period 1990-2021

flood_link = flood_link_original

years = wl_00s.year.unique()

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

flood_link.to_csv('flood_link_GEE_SPA_overall')

### split into lookup tables 

### water level 

SPA_OA_wl = pd.DataFrame()

SPA_OA_wl['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_wl['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_wl['Value'] = flood_link['wl_KK'].astype(float)

SPA_OA_wl.to_excel('SPA_OA_wl.xlsx', index=False)

#### overall 

SPA_OA_OA = pd.DataFrame()

SPA_OA_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_OA_OA.to_excel('SPA_OA_OA.xlsx', index=False)


##### July 15 
SPA_OA_J15 = pd.DataFrame()

SPA_OA_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_OA_J15.to_excel('SPA_OA_J15.xlsx', index=False)


#### July 31

SPA_OA_J31 = pd.DataFrame()

SPA_OA_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_OA_J31.to_excel('SPA_OA_J15.xlsx', index=False)


#### Aug 15 
SPA_OA_A15 = pd.DataFrame()

SPA_OA_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_OA_A15.to_excel('SPA_OA_A15.xlsx', index=False)


#### Aug 31
SPA_OA_A31 = pd.DataFrame()

SPA_OA_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_OA_A31.to_excel('SPA_OA_A31.xlsx', index=False)


#### Sep 25

SPA_OA_S25 = pd.DataFrame()

SPA_OA_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_OA_S25.to_excel('SPA_OA_S25.xlsx', index=False)

flood_link.head()


,wl_KK,date,year,count_reached,freq_overall,freq_Jul15,freq_Jul31,freq_Aug15,freq_Aug31,freq_Sep25
1,6.82,2018-09-03,2018,1,0.0,0.0,0.0,0.0,0.0,0.0
2,6.70,2018-09-13,2018,2,0.2,0.0,0.0,0.0,0.0,0.1
3,6.68,2019-09-12,2019,3,0.2,0.0,0.0,0.0,0.1,0.1
4,6.68,2019-09-13,2019,4,0.2,0.0,0.0,0.0,0.1,0.1
5,6.66,2018-08-19,2018,5,0.2,0.0,0.0,0.0,0.1,0.1


In [28]:
#### implement the function for the OVERALL time period 1990-2021

flood_link = flood_link_original

years = wl_pre08.year.unique()

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 


flood_link.to_csv('flood_link_GEE_SPA_90s')

### split into lookup tables 




### overall 

SPA_90s_OA = pd.DataFrame()

SPA_90s_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_90s_OA.to_excel('SPA_90s_OA.xlsx', index=False)


##### July 15 
SPA_90s_J15 = pd.DataFrame()

SPA_90s_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_90s_J15.to_excel('SPA_90s_J15.xlsx', index=False)


#### July 31

SPA_90s_J31 = pd.DataFrame()

SPA_90s_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_90s_J31.to_excel('SPA_90s_J15.xlsx', index=False)


#### Aug 15 
SPA_90s_A15 = pd.DataFrame()

SPA_90s_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_90s_A15.to_excel('SPA_90s_A15.xlsx', index=False)


#### Aug 31
SPA_90s_A31 = pd.DataFrame()

SPA_90s_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_90s_A31.to_excel('SPA_90s_A31.xlsx', index=False)


#### Sep 25

SPA_90s_S25 = pd.DataFrame()

SPA_90s_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_90s_S25.to_excel('SPA_90s_S25.xlsx', index=False)

flood_link.head()

,wl_KK,date,year,count_reached,freq_overall,freq_Jul15,freq_Jul31,freq_Aug15,freq_Aug31,freq_Sep25
1,6.82,2018-09-03,2018,1,0.105263,0.0,0.0,0.0,0.000000,0.105263
2,6.70,2018-09-13,2018,2,0.263158,0.0,0.0,0.0,0.000000,0.157895
3,6.68,2019-09-12,2019,3,0.263158,0.0,0.0,0.0,0.052632,0.157895
4,6.68,2019-09-13,2019,4,0.263158,0.0,0.0,0.0,0.052632,0.157895
5,6.66,2018-08-19,2018,5,0.263158,0.0,0.0,0.0,0.052632,0.157895


In [66]:
#### implement the function for the 00s time period 1990-2021


flood_link = flood_link_original

years = wl_s.year.unique()

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 


flood_link.to_csv('flood_link_GEE_SPA_00s')
### split into lookup tables 



### overall 

SPA_00s_OA = pd.DataFrame()

SPA_00s_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_00s_OA.to_excel('SPA_00s_OA.xlsx', index=False)


##### July 15 
SPA_00s_J15 = pd.DataFrame()

SPA_00s_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_00s_J15.to_excel('SPA_00s_J15.xlsx', index=False)


#### July 31

SPA_00s_J31 = pd.DataFrame()

SPA_00s_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_00s_J31.to_excel('SPA_00s_J15.xlsx', index=False)


#### Aug 15 
SPA_00s_A15 = pd.DataFrame()

SPA_00s_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_00s_A15.to_excel('SPA_00s_A15.xlsx', index=False)


#### Aug 31
SPA_00s_A31 = pd.DataFrame()

SPA_00s_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_00s_A31.to_excel('SPA_00s_A31.xlsx', index=False)


#### Sep 25

SPA_00s_S25 = pd.DataFrame()

SPA_00s_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_00s_S25.to_excel('SPA_00s_S25.xlsx', index=False)


In [67]:
flood_link.head()

,wl_KK,date,Sentinel_date_PA,year,image_date,count_reached,freq_overall,freq_Jul15,freq_Jul31,freq_Aug15,freq_Aug31,freq_Sep25
1,6.79,2018-08-31,2018-09-12,2018,2018-08-31,1,1.0,0.1,0.1,0.7,0.9,1.0
2,6.71,2018-09-15,2018-09-27,2018,2018-09-15,2,1.0,0.2,0.3,0.8,0.9,1.0
3,6.67,2019-09-15,2019-09-27,2019,2019-09-15,3,1.0,0.2,0.4,0.8,0.9,1.0
4,6.67,2018-08-15,2018-08-27,2018,2018-08-15,4,1.0,0.2,0.4,0.8,0.9,1.0
5,6.66,2018-09-25,2018-10-07,2018,2018-09-25,5,1.0,0.2,0.4,0.8,0.9,1.0


In [13]:
#### implement the function for the OVERALL time period 1990-2021

years = wl_10s.year.unique()

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

flood_link.to_csv('flood_link_GEE_SPA_10s')

### split into lookup tables 



#### overall

SPA_10s_OA = pd.DataFrame()

SPA_10s_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_10s_OA.to_excel('SPA_10s_OA.xlsx', index=False)


##### July 15 
SPA_10s_J15 = pd.DataFrame()

SPA_10s_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_10s_J15.to_excel('SPA_10s_J15.xlsx', index=False)


#### July 31

SPA_10s_J31 = pd.DataFrame()

SPA_10s_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_10s_J31.to_excel('SPA_10s_J15.xlsx', index=False)


#### Aug 15 
SPA_10s_A15 = pd.DataFrame()

SPA_10s_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_10s_A15.to_excel('SPA_10s_A15.xlsx', index=False)


#### Aug 31
SPA_10s_A31 = pd.DataFrame()

SPA_10s_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_10s_A31.to_excel('SPA_10s_A31.xlsx', index=False)


#### Sep 25

SPA_10s_S25 = pd.DataFrame()

SPA_10s_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_10s_S25.to_excel('SPA_10s_S25.xlsx', index=False)



In [14]:
#### implement the function for the OVERALL time period 1990-2021

years = wl_pre08.year.unique()

flood_link.to_csv('flood_link_GEE_SPA_pre08')

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 



### split into lookup tables 



#### overall

SPA_pre08_OA = pd.DataFrame()

SPA_pre08_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_pre08_OA.to_excel('SPA_pre08_OA.xlsx', index=False)


##### July 15 
SPA_pre08_J15 = pd.DataFrame()

SPA_pre08_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_pre08_J15.to_excel('SPA_pre08_J15.xlsx', index=False)


#### July 31

SPA_pre08_J31 = pd.DataFrame()

SPA_pre08_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_pre08_J31.to_excel('SPA_pre08_J15.xlsx', index=False)


#### Aug 15 
SPA_pre08_A15 = pd.DataFrame()

SPA_pre08_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_pre08_A15.to_excel('SPA_pre08_A15.xlsx', index=False)


#### Aug 31
SPA_pre08_A31 = pd.DataFrame()

SPA_pre08_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_pre08_A31.to_excel('SPA_pre08_A31.xlsx', index=False)


#### Sep 25

SPA_pre08_S25 = pd.DataFrame()

SPA_pre08_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_pre08_S25.to_excel('SPA_pre08_S25.xlsx', index=False)




In [15]:
#### implement the function for the OVERALL time period 1990-2021

years = wl_post08.year.unique()

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 


flood_link.to_csv('flood_link_GEE_SPA_post08')
### split into lookup tables 



#### overall

SPA_post08_OA = pd.DataFrame()

SPA_post08_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_post08_OA.to_excel('SPA_post08_OA.xlsx', index=False)


##### July 15 
SPA_post08_J15 = pd.DataFrame()

SPA_post08_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_post08_J15.to_excel('SPA_post08_J15.xlsx', index=False)


#### July 31

SPA_post08_J31 = pd.DataFrame()

SPA_post08_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_post08_J31.to_excel('SPA_post08_J15.xlsx', index=False)


#### Aug 15 
SPA_post08_A15 = pd.DataFrame()

SPA_post08_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_post08_A15.to_excel('SPA_post08_A15.xlsx', index=False)


#### Aug 31
SPA_post08_A31 = pd.DataFrame()

SPA_post08_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_post08_A31.to_excel('SPA_post08_A31.xlsx', index=False)


#### Sep 25

SPA_post08_S25 = pd.DataFrame()

SPA_post08_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_post08_S25.to_excel('SPA_post08_S25.xlsx', index=False)



